# Imports

In [1]:
from pprint import pprint
import regex as re
import html
from bs4 import BeautifulSoup
import requests
import pandas as pd
from helium import *
import praw

import red_downloader # used to scrape reddit images. Credit: JackhammerYT (https://github.com/JackhammerYT/RedDownloader)
import download_imgs # used in getAsyncImgFunctions()

import multiprocessing
from multiprocessing import Process
import pickle
import os
import json
import time

# Global Variables & Functions

In [2]:
dataDir = 'dataset/'
imgTypes = list(filter(lambda v: re.match('\d{1,3}\. ', v), os.listdir(dataDir)))
imgTypes

['0. normal',
 '1. fmemes',
 '2. ememes',
 '3. eSocialMedia',
 '4. fFbPosts',
 '5. fTwtrPosts',
 '6. fTxtMssgs',
 '7. eGreetingAndMisc']

In [5]:
def pklSave(contentToBeSaved, fullPath):
    with open(fullPath, 'wb') as f:
        pickle.dump(contentToBeSaved, f)

def pklLoad(fullPath):
    with open(fullPath, 'rb') as f:
        contentToBeLoaded = pickle.load(f)
    return contentToBeLoaded

## Setting up images and Labels' dictionaries

In [39]:
imgTypeToNum = {}
for  i, type in enumerate(imgTypes):
    imgTypeToNum[type] = i
imgTypeToNum

{'0. normal': 0,
 '1. fmemes': 1,
 '2. ememes': 2,
 '3. eSocialMedia': 3,
 '4. fFbPosts': 4,
 '5. fTwtrPosts': 5,
 '6. fTxtMssgs': 6,
 '7. eGreetingAndMisc': 7}

In [7]:
pklSave(imgTypeToNum, dataDir+"imgTypeToNum.pickle") # to be later used in another .ipynb file

# Scraping Foreign Memes

## From [Kaggle's Meme Generator Dataset](https://www.kaggle.com/datasets/electron0zero/memegenerator-dataset)

### Getting memes' URLs

In [8]:
fmemesCsvLinks = pd.read_csv("dataset/memegenerator.csv", usecols=["Meme ID", "Base Meme Name"]).squeeze() # convert to Series
fmemesCsvLinks.head()

,Meme ID,Base Meme Name
0,10509464,Spiderman Approves
1,12285257,Alright Then Business Kid
2,20612245,Archer
3,20614628,Futurama Fry
4,24194267,One Does Not Simply


Upon inspection, we want to get the original template of memes, not its varientswe'll find that to fetch a page of only the meme image, the URL has to be in this format <br>
`https://memegenerator.net/img/instances/XX.jpg` <br>
Where `XX` is the unique id of the image on [Meme Generator](https://memegenerator.net/) <br>
Thus, we will use the `Meme ID` column

In [9]:
fmemesIds = fmemesCsvLinks['Meme ID']
fmemesIds.head()

0    10509464
1    12285257
2    20612245
3    20614628
4    24194267
Name: Meme ID, dtype: object

In [10]:
pd.set_option('max_colwidth', 70)
fmemesLinks = 'https://memegenerator.net/img/instances/' + fmemesIds + '.jpg'
fmemesLinks.head()

0    https://memegenerator.net/img/instances/10509464.jpg
1    https://memegenerator.net/img/instances/12285257.jpg
2    https://memegenerator.net/img/instances/20612245.jpg
3    https://memegenerator.net/img/instances/20614628.jpg
4    https://memegenerator.net/img/instances/24194267.jpg
Name: Meme ID, dtype: object

In [11]:
pd.set_option('max_colwidth', 40)

### Downloading the foreign images

In [12]:

def getAsyncImgFunctions(imgsLinks, imgType, interval, offset=0):
    fns = []
    i = 0
    check = True
    while i < len(imgsLinks):
        fns.append((download_imgs.downloadImgs, (imgType, imgsLinks, i, i+interval, offset)))
        i += interval
    return fns

In [13]:
fns = getAsyncImgFunctions(fmemesLinks, imgTypes[1], 2000)

In [14]:
# Using runInParallel() below, we will run xx functions parallely to fetch data
print(len(fns))

29


In [15]:
def runInParallel(fns):
  proc = []
  for fn in fns:
    p = Process(target=fn[0], args=(fn[1]))
    p.start()
    proc.append(p)
  for p in proc:
    p.join()

In [16]:
#if __name__ == '__main__':
    #runInParallel(fns)

# Scraping Egyptian Memes

## From Imgur using this [Facebook post]()

In [17]:
fbPostContent = "Templates HQ - المنتدى ( ألبومات تمبلتس _ مستلزمات الكوميك _ شروحات _ افضل المواقع _ مستلزمات التصميم _ ايموشنز _ وشوش ) : plus.google.com/u/0/communities/108675057016512986137 ____ ألبومات تمبلتس :(تمبلتس | مقصوصه ، افلام ، مسلسلات ، مشاهد ، برامج ) ___________ .......... محمد هنيدى .......... رمضان مبروك ابو العالمين حمودة | imgur.com/a/HxfPF اسماعيلية رايح جى | imgur.com/a/i5Msi فول الصين العظيم | imgur.com/a/C8OLI جائنا البيان التالى | imgur.com/a/r4X66 صاحب صاحبه | imgur.com/a/ilzyz يا انا يا خالتى | imgur.com/a/sTx1q امير البحار | imgur.com/a/mRYs0 وش اجرام | imgur.com/a/lhDIg .......... محمد سعد .......... اللي بالي بالك | imgur.com/a/mMJkA اللمبى‏ | imgur.com/a/kNwD8 بوحه | imgur.com/a/nqdmP .......... عادل امام .......... عريس من جهة امنيه | imgur.com/a/h4pA7 السفارة فى العمارة | imgur.com/a/Y519O سلام يا صاحبى | imgur.com/a/XsucN زهايمر | imgur.com/a/iLUE0 .......... احمد مكى .......... لا تراجع ولا استسلام | imgur.com/a/Ri6QP الكبير اوى | imgur.com/a/fUQNf طير انت | imgur.com/a/NPUrK .......... شيكو و هشام ماجد و أحمد فهمي .......... سمير وشهير وبهير | imgur.com/a/UeRnH حملة فريزر | imgur.com/a/bKB5D بنات العم | imgur.com/a/wMQTL .......... كريم عبد العزيز .......... حرامية في تايلاند‏ | imgur.com/a/MxxS6 فى محطة مصر | imgur.com/a/xxLOL الباشا تلميذ | imgur.com/a/VS6Ui ابو على | imgur.com/a/GdBOz .......... احمد رزق و احمد عيد .......... اوعى وشك | imgur.com/a/tw0Zk فيلم ثقافى‏ | imgur.com/a/AIgjT .......... احمد ادم.......... الرجل الابيض المتوسط | imgur.com/a/OKOME معلش إحنا بنتبهدل | imgur.com/a/VFUyL .......... احمد حلمى .......... عسل اسود | imgur.com/a/mXJkz على جثتى | imgur.com/a/1xzZT كده رضا | imgur.com/a/dxfdi .......... حماده هلال .......... الهرم الرابع | imgur.com/a/wAhJp عيال حبيبة |imgur.com/a/QpP7R غبي منه فيه | imgur.com/a/hYvug مقلب حرامية | imgur.com/a/bbTvu هى فوضى | imgur.com/a/PMlpR تيمور وشفيقه | imgur.com/a/qjcBf طباخ الريس | imgur.com/a/vIzBa كابتن هيما | imgur.com/a/X3abp حلم العمر | imgur.com/a/h1w4F الناظر | imgur.com/a/UQWH7 ايظن | imgur.com/a/G4q1vA2 Euc | imgur.com/a/wM1rrTE ..... تمبلتس باسم يوسف | imgur.com/a/QoM3E تمبلتس افلام اجنبى | imgur.com/a/m2anE ـــــــــــــــــــــــــــــــــــــــ البوم الكيف (مشهد العزاء) | imgur.com/a/ysymZ البوم عايز حقى (مشهد هواء هواء) | imgur.com/a/rBvFC البوم مرتضى (انا معايا دورى وصوير و 4 كاس) | imgur.com/a/Mf9OK البوم لخمة راس (هو انت ياد مش بتتكيف غير لم تضرب)| imgur.com/a/5v0GK البوم ابو العربى ( مشهد الكباريه ) | imgur.com/a/Ga7kGAY البوم الناظر (اوعى يغرك جسمك) | imgur.com/a/hwjFu ألبوم لن اعيش فى جلباب ابى | imgur.com/a/K2bosbc ـــــــــــــــــــــــــــــــــــــــ تمبلتس اعلانات ........ اتصالات محمد رمضان 2017 | imgur.com/a/Pxuv9 فودافون 2017 | imgur.com/a/BaCF8 حديد الجارحى | imgur.com/a/4uLwR كارير | imgur.com/a/pxapm بوادى | imgur.com/a/QdWKQ ـــــــــــــــــــــــــــــــــــــــ تمبلتس مقصوصة | imgur.com/a/y1bDn ــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ ⛔ كل الألبومات بنعدلها كل يوم وبنضيف عليها كل التمبلتس الجديده | Facebook"
ememesHashes = re.findall("imgur.com/a/([A-Za-z]{5})", fbPostContent)
print(ememesHashes)

['HxfPF', 'ilzyz', 'lhDIg', 'mMJkA', 'nqdmP', 'XsucN', 'fUQNf', 'NPUrK', 'UeRnH', 'wMQTL', 'xxLOL', 'GdBOz', 'AIgjT', 'OKOME', 'VFUyL', 'mXJkz', 'dxfdi', 'wAhJp', 'hYvug', 'bbTvu', 'PMlpR', 'qjcBf', 'vIzBa', 'ysymZ', 'rBvFC', 'hwjFu', 'pxapm', 'QdWKQ']


### Using [Imgur's API](https://api.imgur.com/)

In [18]:
client_id = 'your-data-here'
client_secret = 'your-data-here'

In [19]:
headers = {
    "Content-Type": "text",
    'Authorization' : "Client-ID " + client_id
}

In [20]:
jsn = json.loads(requests.get('https://api.imgur.com/3/album/i5Msi/images', headers=headers).text)
pprint(jsn)


{'data': {'error': 'Invalid client_id',
          'method': 'GET',
          'request': '/3/album/i5Msi/images'},
 'status': 403,
 'success': False}


Note that initially (before I removed cliend_id string) the JSON response was like this: <br><br>
<img src='project_media/0. imgur_api_response.png' width=350>

In [21]:
ememesLinks = []
def getImgurLinks():
    for hash in ememesHashes:
        imgsDetailsJson = json.loads(requests.get(f'https://api.imgur.com/3/album/{hash}/images', headers=headers).text)
        for img in imgsDetailsJson['data']:
            ememesLinks.append(img['link'])

In [22]:
#getImgurLinks()

In [23]:
#pklSave(ememesLinks, dataDir+"ememesImgurLinks.pickle") # store the links, in order not to keep accessing imgur's API each time we run the cells

In [24]:
ememesLinks = pklLoad(dataDir+"ememesImgurLinks.pickle")

In [25]:
#fns = getAsyncImgFunctions(ememesLinks, imgTypes[2], len(ememesLinks)//50) #last argument means this: if #links to download are 10,000 then there will be 50 functions, each will take from link i to link i + 10,000/50 (which is 200)  
#if __name__ == '__main__':
    #runInParallel(fns)

## From Facebook's public meme pages

In [44]:
fbLinks = [ # These are just some Egyptian pages that were gathered
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%84%D8%A7-%D9%81%D8%A7%D8%A6%D8%AF%D8%A9-%D9%85%D9%86%D9%87%D8%A7-%D9%85%D8%AB%D9%84-%D8%AD%D9%8A%D8%A7%D8%AA%D9%83-290489111460684/photos",
    "https://www.facebook.com/MemesYard/photos/?ref=page_internal",
    "https://www.facebook.com/memes.Stolen.1.0/photos/?ref=page_internal",

    "https://www.facebook.com/%D8%A8%D9%86%D8%B3%D8%B1%D9%82-%D9%85%D9%8A%D9%85%D8%B2-%D9%88%D9%83%D9%88%D9%85%D9%8A%D9%83-%D8%B9%D8%B4%D8%A7%D9%86-%D9%85%D8%B4-%D8%A8%D9%86%D8%B9%D8%B1%D9%81-%D9%86%D8%B9%D9%85%D9%84-1735677470065180/photos/?ref=page_internal",
    "https://www.facebook.com/profile.php?id=100064850389099&sk=photos",
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%84%D9%88%D8%B1%D8%AF-%D9%82%D9%85%D8%AF-100182408618014/photos/?ref=page_internal",
    
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%85%D8%B5%D8%B1%D9%8A%D9%87-101966151287716/photos/?ref=page_internal",
    "https://www.facebook.com/%D9%85%D9%8A%D9%85%D8%B2-%D9%85%D8%B4-%D9%87%D9%8A%D9%81%D9%87%D9%85%D9%87%D8%A7-%D8%A7%D9%84%D9%86%D9%88%D8%B1%D9%85%D9%8A%D8%B2-%D8%B9%D8%B4%D8%A7%D9%86-%D9%86%D9%88%D8%B1%D9%85%D9%8A%D8%B2-833501407023909/photos/?ref=page_internal",
    "https://www.facebook.com/arabicclassicalartmemes/photos/?ref=page_internal",
    
    "https://web.facebook.com/True.Memes.Comics/photos/?ref=page_internal",
    "https://web.facebook.com/societyforsarcasm/photos/?ref=page_internal",
    "https://web.facebook.com/memes.officil/photos",
]

In [27]:
def scrollTillEnd(browser):
    """ 
    This function uses Helium (or Selenium) browser to scroll 
    Through a page that dynamically loads content in order to 
    fully load the html in browser.page_source, example:
    https://www.facebook.com/progressofyear/photos
    """
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    lenOfPage = browser.execute_script("var lenOfPage=document.body.scrollHeight; return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight); window.scrollBy(0,-200);") # "-200" scrolls up a little, so that FB realizes that the user is approaching the end, thus load more content
        lenOfPage = browser.execute_script("var lenOfPage=document.body.scrollHeight; return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

In [28]:
def fbPageName(htmlText):
    """
    Fetches name of a FB page, which upon inspection is usally in 
    the first <h1>, <h2>, <span> of <h1> or <span> of <h2> 
    """
    headerOfPageTitle = re.search('<h[12] .+?(?=/h[12]>)', htmlText)
    if (headerOfPageTitle is None):
        return ""
    headerOfPageTitle = headerOfPageTitle.group() #group() to get the entired matched string out of the regex "match" object
    spanText = re.search('<span>(.*)</span>', headerOfPageTitle)
    if (spanText is not None):
        pageName = spanText.group(1).strip() # group(1) to return what is between parentheses
    else:
        pageName = re.search('>(.*)<', headerOfPageTitle).group(1).strip()
    return pageName

In [29]:
def fbExtractImgsLinks(htmlText):
    """
    Extracts links to images in the "photos" section of a FB page,
    which usually starts from the label "Photos" or "All photos"
    """
    try:
        htmlStartingFromAllPhotos = re.findall(r'>(?:All )?[pP]hotos.*', htmlText)[0] # Regex to get pages starting from "All photos" or "photos", but it has to be preceded by ">", in order not to match "photos" in random URLs
    except Exception as e:
        print(e)
        return []
    imgLinks = re.findall(r'https://scontent[^"]+', htmlStartingFromAllPhotos)
    imgLinks = [html.unescape(link) for link in imgLinks][1:] #avoiding first image, as it might be the page's logo
    return imgLinks

def fbExtractImgsLinksOld(heliumBrowser): # didn't use this, as regex is faster than selenium methods 
    imgTags = heliumBrowser.find_elements_by_tag_name('img')
    imgLinks = []
    for link in imgTags:
        if link.get_attribute('src') is not None:
            imgLinks.append(link.get_attribute('src'))
    imgLinks = [html.unescape(link) for link in imgLinks][1:] #avoiding first image, as it might be the page's logo
    return imgLinks

In [48]:
def fbPageImgScraper(pagesLinks, heliumBrowser, singlePagePickleName, allPagesPickleName, pagesNamesPickle, prevObtainedPages={}):
    """
    Uses fbPageName() to get page name from each FB page in the "pagesLinks" list,
    and  Uses fbExtractImgsLinks() to get all images' links from these pages.
    Note that "prevObtainedPages" is a dictionary of the already fetched pages, so that 
    if this function is executed again, it won't scrape previously scraped pages.
    At the end, it saves all the links and page names in pickle files.
    """
    allLinks = []
    prevObtainedPages = {}
    for i in range(len(pagesLinks)):
        heliumBrowser.get(pagesLinks[i])
        pageName = fbPageName(heliumBrowser.page_source)
        if pageName == "":
            print("page not found...")
            continue
        if (pageName in prevObtainedPages):
            print("This page has already been scraped")
            continue
        time.sleep(1) # code from here till the try/except is to close the popup that sometimes appear when scrolling down a fb page (while not signed in)
        press(PAGE_DOWN) 
        press(PAGE_DOWN)
        time.sleep(1)
        try:
            click('Close')
        except Exception as e:
            print("pop-up didn't  show, moving on...")
        print("Scrolling page:", pageName)
        scrollTillEnd(heliumBrowser)
        imgLinks = fbExtractImgsLinks(heliumBrowser.page_source)
        allEmemesLinks.extend(imgLinks) #extend() expands the elements of an iterable
        pklSave(imgLinks, f"{dataDir}{singlePagePickleName}{i}.pickle")
        prevObtainedPages[pageName] = (i, pagesLinks[i])
        if i == len(pagesLinks)-1:
            print("Finished scraping pages!")
        else:
            print("Finished, scraping next link...", end='\n\n')
    pklSave(allEmemesLinks, f"{dataDir}{allPagesPickleName}.pickle")
    pklSave(prevObtainedPages, f"{dataDir}{pagesNamesPickle}.pickle")

In [32]:
# Side note: as you see, the function will sequentially check the fb pages, that's because my attempt of checking the pages asynchronously in "fb_page_img_scraper.py" have failed
#browser = start_chrome(headless=False)
#fbPageImgScraper(fbLinks, browser, "ememesLinksfbPage", "allFbPagesEmemes", "fbEMemesPagesUsed")
#browser.quit()


Note that when `fbPageImgScraper()` was initially called, it saved the links in .pickle files <br>and printed the name of the pages, such as these last 4: <br><br>
<img src='project_media/1. fbEMemes.png' width=350>

In [34]:
allEmemesLinks = pklLoad(f"{dataDir}allFbPagesEmemes.pickle")
prevObtainedPages = pklLoad(f"{dataDir}fbEMemesPagesUsed.pickle")

In [36]:
print("Total Egyptian memes scraped from FB:", len(allEmemesLinks))
print("Total FB pages scraped:", len(prevObtainedPages))

Total Egyptian memes scraped from FB: 15749
Total FB pages scraped: 12


### Downloading the memes into "`2. ememes`" directory

In [42]:
# Note 1: I've previously saved 1247 memes (from imgur) in that folder
# I want to save files starting from the index after the last saved photo "ememes0001246.jpg"
# So it will start from "ememes0001247.jpg" 
# Note 2: The output of this cell is obviously not 1247, because by now I've already added the rest of the images
prevFiles = os.listdir(f"{dataDir}{imgTypes[2]}")
if len(prevFiles) > 0:
    lastFileName = prevFiles[-1]
    dotLoc = lastFileName.find('.')
    offset = int(lastFileName[dotLoc-7:dotLoc])+1
    print(offset)

1247


In [40]:
#fns = getAsyncFunctions(allEmemesLinks, imgTypes[2], len(allEmemesLinks)//50, offset)
#if __name__ == '__main__':
#    runInParallel(fns)

In [43]:
# to-do: 
# Uncomment and execute the cell above this one
# scrape links of these new folders; one by one

# Scraping Egyptian Social Media Images
For example, screenshots from FB's posts, twitter's tweets, whatsapp's/messenger's chats <br>
Examples of each of these: <br><br>
<img src='project_media/2. eSocialMedia.png' width=600>

In [47]:
fbLinks = [ # These are just some Egyptian pages that were gathered
    'https://www.facebook.com/inbox.ertu/photos/?ref=page_internal',
    'https://www.facebook.com/Inbox.Status/photos',
]

In [49]:
#browser = start_chrome(headless=False)
#fbPageImgScraper(fbLinks, browser, "eSocialMediaLinksfbPage", "allFbPagesESocialMedia", "fbESocialMediaPagesUsed")
#browser.quit()
# to-do: run the above lines ^^

In [ ]:
allESocialMediaLinks = pklLoad(f"{dataDir}allFbPagesESocialMedia.pickle")
prevObtainedPages = pklLoad(f"{dataDir}fbESocialMediaPagesUsed.pickle")

In [ ]:
#fns = getAsyncFunctions(allEmemesLinks, imgTypes[3], len(allESocialMediaLinks)//50)
#if __name__ == '__main__':
#    runInParallel(fns)

# Scraping Foreign Social Media Images
For example, screenshots from FB's posts, twitter's tweets, whatsapp's/messenger's chats <br>
Examples: <br><br>
<img src='project_media/3. fFb.png' width=300>
<img src='project_media/4. fTwtr.png' width=300>
<img src='project_media/5. fWhats.png' width=300>

In [52]:
redditLinks = [
    'https://www.reddit.com/r/FacebookCringe/',
    'https://www.reddit.com/r/bestoftwitter/',
    'https://www.reddit.com/r/texts/',
    'https://www.reddit.com/r/GoodFakeTexts/',
    'https://www.reddit.com/r/Badfaketexts/'
]

## Scraping screenshots of FB posts

In [5]:
red_downloader.DownloadImagesBySubreddit('FacebookCringe', 3000, SortBy='new', quality=360, output='fFbPost', destination=f'{dataDir}{imgTypes[4]}')

Fetching Posts...
Unable to fetch posts
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


## Scraping screenshots of Twitter Tweets

## Scraping screenshots of Chat Messages